In [1]:
import jax.numpy as jnp
import matplotlib.pyplot as plt

import jax.random as jr

from jax.typing import ArrayLike


In [2]:
# Number of layers
K = 5
# Number of neurons per layer
D = 6
# Input layer
D_i = 1
# Output layer
D_o = 1

# Make empty lists
all_weights = [None] * (K+1)
all_biases = [None] * (K+1)


rng = jr.PRNGKey(seed=0)
bias_key,weight_key = jr.split(rng, num=2)

sub_bias_keys = jr.split(bias_key,num=K+1)
sub_weight_keys = jr.split(weight_key,num=K+1)


def init_params(keys,parameter_list:list, D, D_i, D_o,K, is_bias:bool=True)->list:
  for i,sub_key in enumerate(keys):
    if i==0:
      if is_bias:
        parameter_list[i] = jr.normal(sub_key,shape=(D, 1))
      else:
        parameter_list[i] = jr.normal(sub_key,shape=(D, D_i))
    elif i==K:
      if is_bias:
        parameter_list[i] = jr.normal(sub_key,shape=(D_o,1))
      else:
        parameter_list[i] = jr.normal(sub_key,shape=(D_o,D))
    else:
      if is_bias:
        parameter_list[i] = jr.normal(sub_key,shape=(D,1))
      else:
        parameter_list[i] = jr.normal(sub_key,shape=(D,D))
  return parameter_list

all_weights = init_params(sub_weight_keys,all_weights, D, D_i, D_o,K, is_bias=False)
all_biases = init_params(sub_bias_keys,all_biases, D, D_i, D_o,K, is_bias=True)


In [3]:
# Define the Rectified Linear Unit (ReLU) function
def ReLU(preactivation):
  activation = jnp.clip(preactivation,0.0)
  return activation


In [4]:
def compute_network_output(net_input:ArrayLike, all_weights:list[ArrayLike], all_biases:list[ArrayLike]):

  # Retrieve number of layers
  K = len(all_weights) -1

  # We'll store the pre-activations at each layer in a list "all_f"
  # and the activations in a second list[all_h].
  all_f = [None] * (K+1)
  all_h = [None] * (K+1)

  #For convenience, we'll set
  # all_h[0] to be the input, and all_f[K] will be the output
  all_h[0] = net_input

  # Run through the layers, calculating all_f[0...K-1] and all_h[1...K]
  for layer in range(K):
      # Update preactivations and activations at this layer according to eqn 7.16
      # Remmember to use np.matmul for matrrix multiplications
      # TODO -- Replace the lines below
      all_f[layer] = all_biases[layer]+ jnp.matmul(all_weights[layer],all_h[layer])
      all_h[layer+1] = ReLU(all_f[layer])

  # Compute the output from the last hidden layer
  # TO DO -- Replace the line below
  all_f[K] = all_biases[K]+ jnp.matmul(all_weights[K],all_h[K])

  # Retrieve the output
  net_output = all_f[K]

  return net_output, all_f, all_h

In [5]:
# Define in input
net_input = jnp.ones((D_i,1)) * 1.2
# Compute network output
net_output, all_f, all_h = compute_network_output(net_input,all_weights, all_biases)
print("True output = %3.3f, Your answer = %3.3f"%(1.907, net_output[0,0]))

True output = 1.907, Your answer = 21.464


In [6]:
def least_squares_loss(net_output, y):
  return jnp.sum((net_output-y) * (net_output-y))

def d_loss_d_output(net_output, y):
    return 2*(net_output -y);


In [7]:
y = jnp.ones((D_o,1)) * 20.0
loss = least_squares_loss(net_output, y)
print("y = %3.3f Loss = %3.3f"%(y, loss))

y = 20.000 Loss = 2.144


In [10]:
# We'll need the indicator function
def indicator_function(x:ArrayLike):
  x_in = jnp.where(x>=0,1,0)
  return x_in


In [11]:
# Main backward pass routine
def backward_pass(all_weights, all_biases, all_f, all_h, y):
  K = len(all_weights)-1

  # We'll store the derivatives dl_dweights and dl_dbiases in lists as well
  all_dl_dweights = [None] * (K+1)
  all_dl_dbiases = [None] * (K+1)
  # And we'll store the derivatives of the loss with respect to the activation and preactivations in lists
  all_dl_df = [None] * (K+1)
  all_dl_dh = [None] * (K+1)
  # Again for convenience we'll stick with the convention that all_h[0] is the net input and all_f[k] in the net output

  # Compute derivatives of net output with respect to loss
  all_dl_df[K] = jnp.array(d_loss_d_output(all_f[K],y))

  # Now work backwards through the network
  for layer in range(K,-1,-1):
    # TODO Calculate the derivatives of biases at layer this from all_dl_df[layer]. (eq 7.21)
    # NOTE!  To take a copy of matrix X, use Z=np.array(X)
    # REPLACE THIS LINE
    all_dl_dbiases[layer] = all_dl_df[layer]

    # TODO Calculate the derivatives of weight at layer from all_dl_df[K] and all_h[K] (eq 7.22)
    # Don't forget to use np.matmul
    # REPLACE THIS LINE
    all_dl_dweights[layer] = all_h[layer]*all_dl_df[layer]

    # TODO: calculate the derivatives of activations from weight and derivatives of next preactivations (eq 7.20)
    # REPLACE THIS LINE
    all_dl_dh[layer] = jnp.transpose(all_weights[K])


    if layer > 0:
      # TODO Calculate the derivatives of the pre-activation f with respect to activation h (deriv of ReLu function)
      # REPLACE THIS LINE
      all_dl_df[layer-1] = indicator_function(all_f[layer-1])* jnp.matmul(all_dl_dh[layer], all_dl_df[layer])

  return all_dl_dweights, all_dl_dbiases

In [12]:
all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)

TypeError: ignored

In [ ]:
np.set_printoptions(precision=3)
# Make space for derivatives computed by finite differences
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# Let's test if we have the derivatives right using finite differences
delta_fd = 0.000001

# Test the dervatives of the bias vectors
for layer in range(K):
  dl_dbias  = np.zeros_like(all_dl_dbiases[layer])
  # For every element in the bias
  for row in range(all_biases[layer].shape[0]):
    # Take copy of biases  We'll change one element each time
    all_biases_copy = [np.array(x) for x in all_biases]
    all_biases_copy[layer][row] += delta_fd
    network_output_1, *_ = compute_network_output(net_input, all_weights, all_biases_copy)
    network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
    dl_dbias[row] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dbiases_fd[layer] = np.array(dl_dbias)
  print("Bias %d, derivatives from backprop:"%(layer))
  print(all_dl_dbiases[layer])
  print("Bias %d, derivatives from finite differences"%(layer))
  print(all_dl_dbiases_fd[layer])


# Test the derivatives of the weights matrices
for layer in range(K):
  dl_dweight  = np.zeros_like(all_dl_dweights[layer])
  # For every element in the bias
  for row in range(all_weights[layer].shape[0]):
    for col in range(all_weights[layer].shape[1]):
      # Take copy of biases  We'll change one element each time
      all_weights_copy = [np.array(x) for x in all_weights]
      all_weights_copy[layer][row][col] += delta_fd
      network_output_1, *_ = compute_network_output(net_input, all_weights_copy, all_biases)
      network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
      dl_dweight[row][col] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dweights_fd[layer] = np.array(dl_dweight)
  print("Weight %d, derivatives from backprop:"%(layer))
  print(all_dl_dweights[layer])
  print("Weight %d, derivatives from finite differences"%(layer))
  print(all_dl_dweights_fd[layer])
